In [1]:
import os
import time

import utils

import numpy as np

from evo.core import metrics
from evo.tools.file_interface import read_tum_trajectory_file
from evo.core import sync
from evo.tools import plot

import copy

from scipy.spatial.transform import Rotation as R

import matplotlib.pyplot as plt
#%matplotlib inline
%matplotlib tk

import multiprocessing as mp
from functools import partial
import itertools

import pprint

In [2]:
device='AppleVisionPro' #'XReal' # 'Quest3' # 'AVP'
trajName = 'Calib'
root_dir = f"../CalibData/{device}/"

os.chdir(root_dir)
root_dir = "."

## Load trajectories

In [3]:
def align_trajectory(traj_ref, traj_est, offset):
    traj_est_aligned = copy.deepcopy(traj_est)
    traj_ref_aligned = copy.deepcopy(traj_ref)
    traj_ref_aligned, traj_est_aligned = sync.associate_trajectories(traj_ref_aligned, traj_est_aligned, 
                                                                  max_diff=0.05, 
                                                                  offset_2=offset)
    
    traj_est_aligned.align(traj_ref_aligned, correct_scale=False, correct_only_scale=False)#, n=n)
    return traj_ref_aligned, traj_est_aligned

In [4]:
def prepare_traj_pair(traj_gt, traj_xr, offset=0):
    traj_gt = utils.check_monotionic_increaseing(traj_gt)
    traj_gt = utils.check_gt_abnormal_traj(traj_gt, speed_threshold=2.75)
    traj_xr = utils.check_monotionic_increaseing(traj_xr, type='xr')

    traj_gt_cp, traj_xr_cp = align_trajectory(traj_gt, traj_xr, offset=offset)

    fig = utils.plot_trajectory(traj_xr_cp, traj_gt_cp, 
                                benchmark=device, trajectory=trajName, trial="DirectAlign", max_diff=0.05,
                               downsample_rate=0.3)

    return (traj_gt_cp, traj_xr_cp)

In [ ]:
traj_pair_list = []

traj_gt = read_tum_trajectory_file("./Calib/gt.csv")
traj_xr = read_tum_trajectory_file("./Calib/xr.csv")
traj_pair_list.append(prepare_traj_pair(traj_gt, traj_xr, offset=0))

gt - found 0 non-monotonic increasing rows
gt - found 0 abnormal_step_rows
xr - found 0 non-monotonic increasing rows


# Calculate extrincis calibration transformation

In [ ]:
extrinsic_transform = np.eye(4)
extrinsic_transform

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [ ]:
for i, traj_pair in enumerate(traj_pair_list):
    traj_gt_cp, traj_xr_cp = traj_pair[0], traj_pair[1]
    traj_gt_rotate = copy.deepcopy(traj_gt_cp)
    traj_gt_rotate.transform(t=extrinsic_transform, right_mul=True, propagate=False)

    utils.plot_trajectory(traj_xr_cp, traj_gt_rotate, 
                          benchmark=device, trajectory=trajName, trial="AppliedRotation", downsample_rate=0.1)

In [ ]:
def perform_xyz_transform(traj_gt, extrinsic_transform, x, y, z):
    transformation_matrix_test = copy.deepcopy(extrinsic_transform)
    transformation_matrix_test[0,3] = x
    transformation_matrix_test[1,3] = y
    transformation_matrix_test[2,3] = z

    traj_gt_align_test = copy.deepcopy(traj_gt)
    traj_gt_align_test.transform(t=transformation_matrix_test, right_mul=True, propagate=False)
    return traj_gt_align_test, transformation_matrix_test

In [ ]:
def perform_xyz_transform_ape(x, y, z, traj_pair_list, extrinsic_transform):
    transformation_matrix_test = copy.deepcopy(extrinsic_transform)
    transformation_matrix_test[0,3] = x
    transformation_matrix_test[1,3] = y
    transformation_matrix_test[2,3] = z

    ape_list = []
    total_length = 0
    for i, traj_pair in enumerate(traj_pair_list):
        traj_gt_cp, traj_xr_cp = traj_pair[0], traj_pair[1]
        traj_gt_align = copy.deepcopy(traj_gt_cp)
        traj_gt_align.transform(t=transformation_matrix_test, right_mul=True, propagate=False)
        traj_gt_align, traj_xr_cp = align_trajectory(traj_gt_align, traj_xr_cp, offset=0)
        ape_metric = utils.calculate_APE(traj_xr_cp, traj_gt_align, est_offset=0)
        ape = ape_metric.get_statistic(metrics.StatisticsType.rmse)
        ape_list.append(ape*traj_gt_cp.num_poses)
        total_length += traj_gt_cp.num_poses

    avg_ape = np.sum(ape_list)/total_length
    return avg_ape


def parallel_compute(x_array, y_array, z_array, traj_pair_list, extrinsic_transform):
    # Create a partial function with the constant arguments
    partial_compute = partial(perform_xyz_transform_ape, traj_pair_list=traj_pair_list, extrinsic_transform=extrinsic_transform)
    
    # # Combine the individual x, y, z lists into a list of tuples
    # inputs = list(zip(x_list, y_list, z_list))
    # Generate all combinations of x, y, z using itertools.product
    inputs = list(itertools.product(x_array, y_array, z_array))
    
    with mp.Pool(mp.cpu_count()) as pool:
        # Use starmap with the partial function
        results = pool.starmap(partial_compute, inputs)
    
    return results

In [13]:
time.sleep(0.5)

In [ ]:
resolution = 11
x_start, x_end = 0.05, 0.07
y_start, y_end = 0.06, 0.08
z_start, z_end = -0.15, -0.12
#error_matrix = np.ones((resolution,resolution))
x_array = np.linspace(x_start,x_end,resolution)
y_array = np.linspace(y_start,y_end,resolution)
z_array = np.linspace(z_start,z_end,resolution)

# Perform parallel computation
results = parallel_compute(x_array, y_array, z_array, traj_pair_list, extrinsic_transform)

# Print the results
# for (x, y, z), result in zip(itertools.product(x_array, y_array, z_array), results):
#     print(f"compute({x}, {y}, {z}, {constant1}, {constant2}) = {result}")


In [41]:
# Find the minimum result and its corresponding (x, y, z) values
inputs = list(itertools.product(x_array, y_array, z_array))
min_index = np.argmin(results)
min_value = results[min_index]
min_combination = inputs[min_index]
# Print the result
print(f"The combination of x, y, z that gives the minimum result is: {min_combination} with a value of {min_value}")

The combination of x, y, z that gives the minimum result is: (np.float64(0.060000000000000005), np.float64(0.07), np.float64(-0.132)) with a value of 0.010606281661926299


In [ ]:
for i, traj_pair in enumerate(traj_pair_list):
    traj_gt_cp, traj_xr_cp = traj_pair[0], traj_pair[1]
    traj_gt_align = copy.deepcopy(traj_gt_cp)
    #traj_gt_align.transform(t=extrinsic_transform, right_mul=True, propagate=False)
    #plot_trajectory(traj_xr_cp, traj_gt_align, benchmark=None, trajectory=None, trial=None)

    traj_gt_align_test, transformation_matrix_test = perform_xyz_transform(traj_gt_align, extrinsic_transform, 
                                                                        min_combination[0], min_combination[1], min_combination[2])
    traj_gt_align_test, traj_xr_cp = align_trajectory(traj_gt_align_test, traj_xr_cp, offset=0)

    utils.plot_trajectory(traj_xr_cp, traj_gt_align_test, benchmark=device, trajectory=trajName, trial="AppliedTransform")

In [43]:
transformation_matrix_test

array([[ 1.   ,  0.   ,  0.   ,  0.06 ],
       [ 0.   ,  1.   ,  0.   ,  0.07 ],
       [ 0.   ,  0.   ,  1.   , -0.132],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])

In [44]:
#utils.plot_trajectory(traj_xr_cp, traj_gt_align_test, benchmark=device, trajectory=trajName, trial="AppliedTransform")

In [45]:
rpe = utils.calculate_RE(traj_xr_cp, traj_gt_align_test, est_offset=0)
np.mean(rpe.error)

np.float64(0.004309076899335913)

In [46]:
ape_metric = utils.calculate_APE(traj_xr_cp, traj_gt, est_offset=0)

In [47]:
ape_metric.get_statistic(metrics.StatisticsType.rmse)

0.16123140051275742

In [48]:
ape_metric = utils.calculate_APE(traj_xr_cp, traj_gt_align_test, est_offset=0)

In [49]:
ape_metric.get_statistic(metrics.StatisticsType.rmse)

0.010606281661926299

In [50]:
result_np = np.array(results).reshape((resolution, resolution, resolution))

In [51]:
# Find the indices of the given number
x_index = np.where(x_array == min_combination[0])[0][0]
y_index = np.where(y_array == min_combination[1])[0][0]
z_index = np.where(z_array == min_combination[2])[0][0]


In [52]:
plt.figure()
plt.plot(result_np[:, y_index, z_index], label="x-axis")
plt.plot(result_np[x_index, :, z_index], label="y-axis")
plt.plot(result_np[x_index, y_index, :], label="z-axis")
plt.legend()
plt.show()

/home/i3t-sigma/miniconda3/envs/xreva/lib/python3.13/tkinter/__init__.py:862: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or Axes decorations smaller.
  func(*args)
